In [ ]:
!pip install selenium

In [ ]:
!pip install requests beautifulsoup4

# **Projeto Final - Big Data e computação em nuvem**
### Webscrapping do site da [National Centers for Environmental Information (NCEI)](https://https://www.ncei.noaa.gov/)

Para a realização do projeto, foi realizado a raspagem de dados do site nacional do estados unidos para a obtenção de dados geográficos, relacionado ao clima e precipitação.

No site, há dados de clima de todos os 49 estados americanos desde 1895. Para a utilização do projeto, foram obtidos os dados de precipitação e temperatura média do perído de 2009 à 2018.

In [ ]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import random
import matplotlib.pyplot as plt
import IPython
import itertools
import numpy as np
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

### **Criação do driver**
Para este projeto será utilizado o *chromedriver* como o agente de consulta do site, para realizar a raspagem de dados.

In [ ]:
#Lista de users agents para evitar o bloqueio do driver
user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
]

#Argumentos do driver
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('--disable-blink-features=AutomationControlled')
options.headless = True

#Escolha do user_agent
user_agent = {'User-Agent': random.choice(user_agents_list)}

options.add_argument('user-agent={0}'.format(user_agent))

driver = webdriver.Chrome(options=options)

### **Funções de *scrapping***

Antes de realizar a raspagem de dados, foram criadas duas funções para facilitar o processo, além de melhorar a compreensão dos códigos.

As funções criadas foram:

- *find_state*: Recebe uma variável do BeautifulSoup e realiza a procura do campo que apresenta o estado em que se apresenta o site. A função retorna uma string com o nome do estado.

- *find_info*: Recebe uma variável do BeautifulSoup e realiza a procura das linhas da tabela apresentada no site onde resgata o ano, o mês e o dado apresentado na tabela. A função retorna uma lista de todos Anos e meses com os dados desejados.

In [ ]:
def find_state(soup):
  """ Encontra o estado em que se apresenta as informações
  Retorna uma string do estado"""

  form = soup.find("div", id="form-sections")
  form_loc = form.find("select", id="location")
  state = form_loc.find("option", selected="selected").text

  return state

def find_info(soup):
  """ Encontra o parâmetro presente na tabela do site, assim como mês e ano da observação
  Retorna uma lista com 3 valores por observação ([Ano, Mês, Parâmetro])
  Obs: O parâmetro é convertido para float
  """
  info = []

  table = soup.find("tbody")
  rows = table.find_all("tr", role="row")

  for row in rows:
    date = row.find_all("td")[0].text.split()
    mth = date[0]
    year = date[1]

    data = row.find_all("td")[1].text
    data_float = float(re.findall(r"\d+.\d+", data)[0])

    info.append([year, mth, data_float])

  return info

### **Webscrapping**
Com as funções que realizam a aquisição dos dados e o driver criado, é feito a operação de webscrapping.

O site da NCEI é um site dinâmico, ou seja, alguns recursos dele são criados dependendo de algum parâmetro dentro do próprio site. No caso o objeto dinâmico seria a tabela, que muda de acordo com a mudança de parâmetros do site, que seria o objeto que seria adquirido.



In [ ]:
# Número de estados
len_states = 49

#Parâmetros que vão ser adquiridos
param = ["tavg", "pcp"]

#Criação do dataframe
df = pd.DataFrame()

for state_num in range(1,len_states + 1,1):

  #Condição para incluir o Alaska
  if state_num == 49:
    state_num = 50

  for p in param:

        #Criação de um dataframe temporário para o loop
        df_tmp = pd.DataFrame()

        time.sleep(1)

        #Url da página com os
        page_url = "https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/statewide/time-series/"+str(state_num)+"/"+p+"/all/10/2009-2018"

        driver.get(page_url)
        driver.implicitly_wait(30)

        # Espera o elemento aparecer na página
        element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/main/div/div/div[6]/table/tbody/tr[1]"))
        )

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        state = find_state(soup)
        info = find_info(soup)

        df_tmp["Year"] = [i[0] for i in info]
        df_tmp["Month"] = [i[1] for i in info]
        df_tmp[p] = [i[2] for i in info]
        df_tmp["State"] = state

        #União do dataframe temporário do loop com o dataframe principal
        df = pd.concat([df, df_tmp], ignore_index=True, axis=0)

df = df.groupby(['State', 'Year', 'Month']).apply(lambda group: group.ffill().bfill())

<ipython-input-113-2b7a6722c76d>:40: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby(['State', 'Year', 'Month']).apply(lambda group: group.ffill().bfill())


### **Exportação dos dados**
Feita a operação de webscrapping, o dataframe é exportado para um arquivo csv e assim pode ser utilizado no projeto.

In [ ]:
from google.colab import files

#Reordenação das colunas
df = df[['State', 'Year', 'Month', 'tavg', 'pcp']]

#Conversão do dataframe para csv e download
df.to_csv('df_weather.csv', index=False)
files.download('df_weather.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>